In [5]:
import numpy as np
import pandas as pd
import seaborn as sns

from pandas.plotting import scatter_matrix
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from time import time
from IPython.display import display

%matplotlib inline

### Load in the data and display some

In [11]:
fAndMVP_data = pd.read_csv("NBA Finals and MVP.csv", delimiter=',')
print(fAndMVP_data.columns)
NBA_data = pd.read_csv("NBA_Dataset.csv")
# display(fAndMVP_data.head(n=5))
# display(NBA_data.head(n=10))

Index(['index', 'Year', 'Western Champion', 'Eastern Champion', 'Result',
       'NBA Champion', 'NBA Vice-Champion', 'Final Sweep ?', 'MVP Name',
       'MVP Height (m)', 'MVP Height (ft)', 'MVP Position', 'MVP Team',
       'MVP Nationality', 'MVP status'],
      dtype='object')


In [18]:
# check for nulls and replace if needed
NBA_data.fillna({'fg_pct': -1.0}, inplace=True)
NBA_data.fillna({'fg3_pct': -1.0}, inplace=True)
NBA_data.fillna({'fg2_pct': -1.0}, inplace=True)
NBA_data.fillna({'efg_pct': -1.0}, inplace=True)
NBA_data.fillna({'ft_pct': -1.0}, inplace=True)
NBA_data.isnull().any()


season              False
player              False
pos                 False
age                 False
team_id             False
g                   False
gs                  False
mp_per_g            False
fg_per_g            False
fga_per_g           False
fg_pct              False
fg3_per_g           False
fg3a_per_g          False
fg3_pct             False
fg2_per_g           False
fg2a_per_g          False
fg2_pct             False
efg_pct             False
ft_per_g            False
fta_per_g           False
ft_pct              False
orb_per_g           False
drb_per_g           False
trb_per_g           False
ast_per_g           False
stl_per_g           False
blk_per_g           False
tov_per_g           False
pf_per_g            False
pts_per_g           False
mp                  False
per                  True
ts_pct               True
fg3a_per_fga_pct     True
fta_per_fga_pct      True
orb_pct              True
drb_pct              True
trb_pct              True
ast_pct     

In [19]:
# check for nulls
fAndMVP_data.isnull().any()

index                False
Year                 False
Western Champion     False
Eastern Champion     False
Result               False
NBA Champion         False
NBA Vice-Champion    False
Final Sweep ?        False
MVP Name             False
MVP Height (m)       False
MVP Height (ft)      False
MVP Position         False
MVP Team             False
MVP Nationality      False
MVP status           False
dtype: bool

In [20]:
# get the number of players in each dataset
n_players_fmvp = fAndMVP_data.shape[0]
n_players_NBA = NBA_data.shape[0]

69
17697


In [22]:
# how to correlate data that is a mix of floats, strings and bools?

ValueError: could not convert string to float: 'none'